# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys
import json
import time

module_path = os.path.abspath(os.path.join('..'))
#Include api directory in path
if module_path not in sys.path:
    sys.path.append(module_path + "\\api")

# Import API key
from api_keys import g_key

output_data_file = "../output_data/"

places_base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [35]:
print(output_data_file + "cities.csv")
df = pd.read_csv(output_data_file + "cities.csv")
df.head()

#Convert Object Types to String
df["City"] = df["City"].astype("string") 
df["Country"] = df["Country"].astype("string")

#Drop index column saved to csv
df = df.iloc[:,1:]
df.head()



../output_data/cities.csv


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kodiak,57.7900,-152.4072,30.20,64,1,6.91,US,1612148328
1,muros,42.7762,-9.0603,57.99,84,88,8.01,ES,1612148599
2,cayenne,4.9333,-52.3333,80.60,78,75,10.36,GF,1612148411
3,ushuaia,-54.8000,-68.3000,50.00,71,75,20.71,AR,1612148490
4,cape town,-33.9258,18.4232,64.00,72,20,21.85,ZA,1612148476


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [60]:
#Setup Ideal Weather Conditions Parameters
min_temp = 70
max_temp = 82
min_windspeed = 0
max_windspeed = 15
min_cloudiness = 0
max_cloudiness = 20
max_humidity = 80

hotel_df = df[df["Max Temp"].between(min_temp,max_temp) & df["Wind Speed"].between(min_windspeed,max_windspeed) &
           df["Cloudiness"].between(min_cloudiness,max_cloudiness)]

hotel_df.insert(1,"Hotel Name"," ")

hotel_df.head()


,City,Hotel Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,lokoja,,7.8024,6.7430,74.35,52,0,3.27,NG,1612148600
7,saint-philippe,,-21.3585,55.7679,80.60,61,0,10.36,RE,1612148329
8,grand gaube,,-20.0064,57.6608,80.01,80,8,4.00,MU,1612148601
11,busselton,,-33.6500,115.3333,82.00,43,0,7.00,AU,1612148602
15,georgetown,,-7.9333,-14.4167,76.71,84,0,12.88,SH,1612148603


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [64]:


for index, row in hotel_df.iterrows():
    location = str(row["Lat"]) + "," + str(row["Lng"])
    params = {"location": location,
              "radius":5000,
              "type":"lodging",
              "language":"en",
              "key": g_key}
    response = requests.get(places_base_url,params=params).json()
    #print(response["results"][0]["name"])
    time.sleep(1)
    try:
        hotel_df.loc[index,"Hotel Name"] = response["results"][0]["name"]
    except:
        print(f'{row["City"]} (row["Country"]) - No Hotel Found' )


7.8024,6.742999999999999


D:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


-21.3585,55.7679
-20.0064,57.6608
-33.65,115.3333
-7.9333,-14.4167
16.1307,119.7824
-35.1667,173.1667
-13.7,-76.2167
19.7297,-155.09
14.8961,-24.4956
6.1241,81.1185
-14.1667,-66.8833
10.5669,72.642
-25.1667,45.05
beloha (row["Country"]) - No Hotel Found
20.0431,-77.5828
10.2605,-85.5851
12.25,109.1833
-6.9811,-34.8339
-5.3166,119.3661
16.8634,-99.8901
-9.4467,-46.3003
-30.3833,153.1
-19.9842,57.6142
18.65,79.6667
17.1612,104.1472
16.5167,-98.75
16.9833,-89.9
16.3167,-98.0167
-19.9309,-60.7718
-22.92,-42.5103
16.2923,119.8062
2.8197,-60.6733


In [65]:
hotel_df

,City,Hotel Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,lokoja,Hotel Charvid,7.8024,6.7430,74.35,52,0,3.27,NG,1612148600
7,saint-philippe,Hôtel L'Ecrin,-21.3585,55.7679,80.60,61,0,10.36,RE,1612148329
8,grand gaube,The Ravenala Attitude Hotel,-20.0064,57.6608,80.01,80,8,4.00,MU,1612148601
11,busselton,Pullman Bunker Bay Resort Margaret River Region,-33.6500,115.3333,82.00,43,0,7.00,AU,1612148602
15,georgetown,Scout's Beach Hut Campsite,-7.9333,-14.4167,76.71,84,0,12.88,SH,1612148603
40,aloleng,Olana Bed & Breakfast,16.1307,119.7824,80.35,56,0,8.16,PH,1612148609
42,ahipara,Ahipara Holiday Park,-35.1667,173.1667,71.02,56,2,7.52,NZ,1612148358
78,pisco,DoubleTree Resort by Hilton Paracas - Peru,-13.7000,-76.2167,70.00,73,0,10.36,PE,1612148360
83,hilo,Hilo Hawaiian Hotel,19.7297,-155.0900,77.00,61,1,8.05,US,1612148599
102,sao filipe,Tortuga B&B,14.8961,-24.4956,71.78,74,0,5.44,CV,1612148625


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
